# Galaxy Classifiers (Spiral, Elliptical, Odd objects)

Here, we developed five machine learning classifiers to identify galaxy images into Spiral, Elliptical, and Odd objects (e.g., ring, lens, disturbed, irregular, merger, and dust lane) that collected from [Galaxy Zoo 2](https://data.galaxyzoo.org/#section-7) (GZ2) cataloge. Totall of 11735 samples including 6139, 4077, 1519 for spiral, elliptical, and odd objects where used in the classifiers.

- Two classifier models including supprot vector machine (SVM) and classic convolutional nueral network (CNN) have designed to use the zernike moments that extracted from original galaxies images. 
- Three classifier models including CNN-Vision Transformer, ResNet50, VGG16 have investigated to work the information of original galaxies images.

### Import libraries:

The list of requried libraries are sklearn, pandas, numpy, tensorflow, matplotlib, etc. 

In [ ]:
# import packages
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.metrics import (roc_curve, roc_auc_score, auc, log_loss,
                             precision_score, recall_score, f1_score,
                             accuracy_score, classification_report,
                             ConfusionMatrixDisplay, confusion_matrix)


import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

#Tensorflow
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.layers import (Dense, Dropout,BatchNormalization, Input, Conv1D, Flatten,
                             MaxPooling1D)
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

import cv2
import os
from skimage.measure import label
from skimage.transform import resize
from skimage.segmentation import slic, mark_boundaries
from skimage.color import label2rgb

#Scikit_learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_curve, roc_auc_score, auc, log_loss,
                             precision_score, recall_score, f1_score,
                             accuracy_score, classification_report,
                             ConfusionMatrixDisplay, confusion_matrix)
from sklearn import metrics
from sklearn.preprocessing import label_binarize

- Using the **"watershed"** algorithm, we cropped the centeral galaxy features.

In [ ]:
def seg_1(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV +cv2.THRESH_OTSU, cv2.THRESH_TRUNC)
  kernel = np.ones((3,3),np.uint8)
  opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
  sure_bg = cv2.dilate(opening,kernel,iterations=3)
  dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
  ret, aa = cv2.threshold(dist_transform,0.01*dist_transform.max(),255,0)
  x,y=np.where(aa==0)
  im=thresh.copy()*0
  im[x,y]=1
  im1 = label(im)
  f=im1[212,212]
  x,y=np.where(im1==f)
  imd=im1.copy()*0
  imd[x,y]=1
  return x,y,imd


def seg_2(img):
  segments = slic(img,n_segments=25,compactness=10)
  ss=label2rgb(segments,img,kind = 'avg')
  aa=ss[:,:,2]
  f=aa[212,212]
  x,y=np.where(aa==f)
  dd=aa.copy()*0
  dd[x,y]=1
  return x,y,dd


def seg_3(img,x,y):
    x1=np.min(x)-20
    x2=np.max(x)+20
    y1=np.min(y)-20
    y2=np.max(y)+20
    imn=img[x1:x2,y1:y2]
    newsize=(x2-x1,x2-x1)
    resized_im =resize(imn,newsize)
    xx = x1 + (x2 - x1) / 2 - 212
    yy = y1 + (y2 - y1) / 2 - 212
    return xx, yy,resized_im


In [ ]:
image_path = r'/path/to/repository/Data/galaxy/image/spiral'

image_files = [os.path.join(image_path, filename) for filename in os.listdir(image_path) if filename.endswith('.jpg')]
cropped_imgs = []
cropped_image_count = 0

for im in image_files:
    xx=100
    yy=100
    img = cv2.imread(im)
    x, y, imd = seg_1(img)  

    if np.min(x)>20 and np.min(y)>20 and np.max(y)<212*2-20 and np.max(x)<212*2-20:
        xx,yy,resized_im=seg_3(img,x,y)
        if (np.abs(xx) >= 30) and (np.abs(yy) >= 30):
            x, y, dd = seg_2(img)
            xx,yy,resized_im=seg_3(img,x,y)
    else: 
        x, y, dd = seg_2(img) 
        if np.min(x)>20 and np.min(y)>20 and np.max(y)<212*2-20 and np.max(x)<212*2-20:
            xx,yy,resized_im=seg_3(img,x,y)      
            # print(xx,yy)
    if (np.abs(xx) < 30) and (np.abs(yy) < 30):
            ms = np.shape(resized_im)
            if ms[0] < 150:
                crop_img = np.array(resize(resized_im, (150, 150)))
            elif (ms[0] < 250) and (ms[0] >= 150):
                crop_img = np.array(resize(resized_im, (250, 250)))
            elif (ms[0] < 350) and (ms[0] >= 250):
                crop_img = np.array(resize(resized_im, (350, 350)))
            elif ms[0] >= 350:
                crop_img = np.array(resize(resized_im, (450, 450)))
                                                                  
            arr_img = np.array(crop_img)
            cropped_imgs.append(arr_img)

            cropped_image_count += 1


print(f"Total number of cropped images: {cropped_image_count}")


## Two classifier models based on Zernike moments (ZMs):

### Compute the ZMs:

##### First we need to compute ZMs for galaxy and non-galaxy images. The ZEMO python package [https://pypi.org/project/ZEMO/] [https://github.com/hmddev1/ZEMO] can be used to compute Zernike moments (ZMs) for images. This package was described in the research paper [[IAJJ](https://ijaa.du.ac.ir/article_374_ad45803d737b0a7d4fc554a244229df6.pdf)].

*Note: The galaxy and non-galaxy images are in RGB format. Here, we used the R channel of images. The size of original Galaxy Zoo 2 images is (424, 424) pixels, while we resized them to (200, 200) pixels. To compute ZMs we set the maximum order number $P{max} = 45$.* 

In [ ]:
from ZEMO import zemo
import cv2

def calculate_zernike_moments(data_dir, image_size, zernike_order):
        
        ZBFSTR = zemo.zernike_bf(image_size, zernike_order, 1)
        
        image_files = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if filename.endswith('.jpg')]
        
        zernike_moments = []
    
        for img_path in image_files:
            image = cv2.imread(img_path)
            resized_image = cv2.resize(image, (image_size,image_size))
            im = resized_image[:, :, 0]
            Z = np.abs(zemo.zernike_mom(np.array(im), ZBFSTR))
            zernike_moments.append(Z)
        
        df = pd.DataFrame(zernike_moments)
    
        return df

- Download **Data** files from [here](https://drive.google.com/file/d/1wxmYQ8qpgaVDuD3kTeBrZlyny0IBA9wn/view?usp=drive_link)

- The directoies of Spiral, Elliptical, Odd objects images:

        - Spiral: /repository/Data/galaxy/image/cropped_spiral
        - Elliptical: /repository/Data/galaxy/image/cropped_elliptical
        - odd objects: /repository/Data/galaxy/image/cropped_odd

In [ ]:
spath = r'/path/to/repository/Data/galaxy/image/cropped_spiral' 
epath = r'/path/to/repository/Data/galaxy/image/cropped_elliptical' 
opath = r'/path/to/repository/Data/galaxy/image/cropped_odd' 

# Defult image size and zernike order. 
image_size = 200
zernike_order = 45

spiral_zm_df = calculate_zernike_moments(spath, image_size, zernike_order)
spiral_zm_df.to_csv('/path/to/repository/Data/galaxy/ZMs/spiral_zms.csv')

elliptical_zm_df = calculate_zernike_moments(epath, image_size, zernike_order)
elliptical_zm_df.to_csv('/path/to/repository/Data/galaxy/ZMs/elliptical_zms.csv')

odd_zm_df = calculate_zernike_moments(opath, image_size, zernike_order)
odd_zm_df.to_csv('/path/to/repository/Data/galaxy/ZMs/odd_zms.csv')

np.shape(spiral_zm_df), np.shape(elliptical_zm_df), np.shape(odd_zm_df)


*Note: Computing of ZMs for above mentioned galaxy and non-galaxy images are slightly consuming time. So, we upladed the zernike moments of both classes in this repository.*

**To load the ZMs please use:**


In [ ]:
spiral_data = pd.read_csv('/path/to/repository/Data/galaxy/ZMs/spiral_zms.csv')
elliptical_data = pd.read_csv('/path/to/repository/Data/galaxy/ZMs/elliptical_zms.csv')
odd_data = pd.read_csv('/path/to/repository/Data/galaxy/ZMs/odd_zms.csv')

spiral_data.drop("Unnamed: 0", axis = 1, inplace = True)
elliptical_data.drop("Unnamed: 0", axis = 1, inplace = True)
odd_data.drop("Unnamed: 0", axis = 1, inplace = True)

all_zm_data = np.concatenate([spiral_data, elliptical_data, odd_data])
np.shape(all_zm_data)

- We use **"0"** for spiral class labels, **"1"** for elliptical class labels, and **"2"** for odd objects class labels.  

In [ ]:
spiral_label = [0] * len(spiral_data)
elliptical_label = [1] * len(elliptical_data)
odd_label = [2] * len(odd_data)

all_labels = np.concatenate([spiral_label, elliptical_label, odd_label])
len(all_labels)

### **SVM + ZMs**

- We split the data set into 75 percent traning set and 25 percent test set:

In [ ]:
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_zm_data, all_labels, np.arange(len(all_labels)), 
                                                                                 test_size=0.25, shuffle=True, random_state=None)

- Since, the galaxy classifiers are unbalance class models, so we used the class weight in the program:

In [ ]:
class_weights = {0: len(all_zm_data) / (3*len(spiral_data)), 1: len(all_zm_data) / (3*len(elliptical_data)), 2: len(all_zm_data) / (3*len(odd_data))}

- The **SVM** model uses radial base kernel (rbf), C = 1.5, and gamma = 'scale' to fit the model on the training set:

In [ ]:
model = SVC(kernel='rbf', probability=True, C=1.5, gamma='scale',class_weight=class_weights)
gz2_training_model = model.fit(X_train, y_train)

- Now, we apply the test set to examine the classification algorithm. Using the predicted label by the machine on original labels, we compute the elements of the confusion matrix.

In [ ]:
y_pred = model.predict(X_test)

con = metrics.confusion_matrix(y_test, y_pred)
print(con)

- To compare the performace of classifier with the random classifier, we calculate the **reciver operation charecterstic curve (ROC curve)**. The **area under the curve (AUC)** shows the probability of True positive rates of the classifier.

In [ ]:
y_score = gz2_training_model.predict_proba(X_test)  # Use predicted probabilities for ROC curve
y_test_bin = label_binarize(y_test, classes=list(range(3)))  # Binarize the true labels

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

class_names = ['Spiral', 'Elliptical', 'Odd objects']

fig, ax = plt.subplots(figsize=(10, 8))

mean_fpr = np.linspace(0, 1, 100)

for j in range(3):  # Iterate through each class
    tprs_interp = []
    for i in range(len(fpr)):
        tprs_interp.append(np.interp(mean_fpr, fpr[i][j], tpr[i][j]))

    mean_tpr = np.mean(tprs_interp, axis=0)
    mean_auc = np.trapz(mean_tpr, mean_fpr)

    ax.plot(mean_fpr, mean_tpr, label=f'{class_names[j]} (area = {mean_auc:.2f})', lw=2)

ax.plot([0, 1], [0, 1], linestyle='--', color='black', label='Random Guess')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title(f'Receiver Operating Characteristic (ROC) Curve')
ax.legend(loc='lower right')

plt.tight_layout()
plt.show()


- To measure the performance metrics of classifier, we compute **(Recall, Precision, f1_score, Accuracy, TSS(True Skill Statistic))**

In [ ]:
recall = recall_score(y_test, y_pred, average= 'weighted')
precision = precision_score(y_test, y_pred, average= 'weighted')
f1_score = f1_score(y_test, y_pred, average= 'weighted')
accuracy = accuracy_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tss=(tp/(tp+fn))-(fp/(fp+tn))

In [ ]:
print("Recall:", recall)
print("Precision:", precision)
print("F1_score:", f1_score)
print("Accuracy:", accuracy)
print("TSS:", tss)

### **1D_CNN + ZMs**

- We use **"0"** for spiral class labels, **"1"** for elliptical class labels, and **"2"** for odd objects class labels.  

In [ ]:
spiral_label = [0] * len(spiral_data)
elliptical_label = [1] * len(elliptical_data)
odd_label = [2] * len(odd_data)

all_labels = np.concatenate([spiral_label, elliptical_label, odd_label])
len(all_labels)

- We split the data set into 75 percent traning set and 25 percent test set:

In [ ]:
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_zm_data, all_labels, np.arange(len(all_labels)), 
                                                                                 test_size=0.25, shuffle=True, random_state=None)

y_train_encoded = to_categorical(y_train, num_classes=3)

- Since, the galaxy classifiers are unbalance class models, so we used the class weight in the program:

In [ ]:
class_weights = {0: len(all_zm_data) / (3*len(spiral_data)), 1: len(all_zm_data) / (3*len(elliptical_data)), 2: len(all_zm_data) / (3*len(odd_data))}

- Due to one dimentional structure of ZMs, we used one dimentional achitecture of CNN: 

In [ ]:
# input value
x = Input(shape=(all_zm_data.shape[1],1))

#hidden layers
c0 = Conv1D(256, kernel_size=3, strides=2, padding="same")(x)
b0 = BatchNormalization()(c0)
m0 = MaxPooling1D(pool_size=2)(b0)
d0 = Dropout(0.1)(m0)

c1 = Conv1D(128, kernel_size=3, strides=2, padding="same")(d0)
b1 = BatchNormalization()(c1)
m1 = MaxPooling1D(pool_size=2)(b1)
d1 = Dropout(0.1)(m1)

c2 = Conv1D(64, kernel_size=3, strides=2, padding="same")(d1)
b2 = BatchNormalization()(c2)
m2 = MaxPooling1D(pool_size=2)(b2)
d2 = Dropout(0.1)(m2)

f = Flatten()(d2)

# output
de0 = Dense(64, activation='relu')(f)
de1 = Dense(32, activation='relu')(de0)
de2 = Dense(2, activation='softmax')(de1)

model = Model(inputs=x, outputs=de2, name="cnn_zm_45_galaxy_nonegalaxy")
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
model.summary()

- The **1D_CNN** model uses EarlyStopping as callback function, batch size = 64, and number of epochs = 30 to fit the model on the training set:

In [ ]:
# Callback Function
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

b_size = 64
e_num = 30

history = model.fit(
X_train, y_train_encoded,
batch_size=b_size,
epochs=e_num,
class_weight=class_weights,
verbose = 1,
callbacks=es,
validation_split=0.1)

- Now, we apply the test set to examine the classification algorithm. Using the predicted label by the machine on original labels, we compute the elements of the confusion matrix.

In [ ]:
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

con = metrics.confusion_matrix(y_test, y_pred_labels)
print(con)

- To compare the performace of classifier with the random classifier, we calculate the **reciver operation charecterstic curve (ROC curve)**. The **area under the curve (AUC)** shows the probability of True positive rates of the classifier.

In [ ]:
y_score = gz2_training_model.predict_proba(X_test)  # Use predicted probabilities for ROC curve
y_test_bin = label_binarize(y_test, classes=list(range(3)))  # Binarize the true labels

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

class_names = ['Spiral', 'Elliptical', 'Odd objects']

fig, ax = plt.subplots(figsize=(10, 8))

mean_fpr = np.linspace(0, 1, 100)

for j in range(3):  # Iterate through each class
    tprs_interp = []
    for i in range(len(fpr)):
        tprs_interp.append(np.interp(mean_fpr, fpr[i][j], tpr[i][j]))

    mean_tpr = np.mean(tprs_interp, axis=0)
    mean_auc = np.trapz(mean_tpr, mean_fpr)

    ax.plot(mean_fpr, mean_tpr, label=f'{class_names[j]} (area = {mean_auc:.2f})', lw=2)

ax.plot([0, 1], [0, 1], linestyle='--', color='black', label='Random Guess')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title(f'Receiver Operating Characteristic (ROC) Curve')
ax.legend(loc='lower right')

plt.tight_layout()
plt.show()


- To measure the performance metrics of classifier, we compute **(Recall, Precision, f1_score, Accuracy, TSS(True Skill Statistic))**

In [ ]:
recall = recall_score(y_test, y_pred, average= 'weighted')
precision = precision_score(y_test, y_pred, average= 'weighted')
f1_score = f1_score(y_test, y_pred, average= 'weighted')
accuracy = accuracy_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tss=(tp/(tp+fn))-(fp/(fp+tn))

In [ ]:
print("Recall:", recall)
print("Precision:", precision)
print("F1_score:", f1_score)
print("Accuracy:", accuracy)
print("TSS:", tss)

## Three classifier models based on the original images:
  
- (Vision Transformer used as data augmentation tool on the Galaxy and Non-Galaxy images.)

### Import the requried libraries:

In [ ]:
# import packages 

import cv2
import os
import numpy as np
import random
from PIL import Image
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split

#Tensorflow
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Dense, Dropout, Input,Conv2D, Flatten,
                             MaxPooling2D,BatchNormalization)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50, VGG16
from keras.callbacks import EarlyStopping

#Torch
import torch
import torch.nn as nn
from torchvision import transforms

- To read the images of each class and convert to Pillow images we used the following function:

In [ ]:
def load_galaxy_images(data_dir, target_size):
        
        """
        Loads, resizes, and processes all JPG images from the specified directory.

        Parameters:
        data_dir (str): The directory containing the JPG images to be processed.
        target_size (tuple): The target size for resizing the images, specified as (width, height).

        Returns:
        list: A list of PIL Image objects, each representing a resized and processed image.

        The function performs the following steps:
        1. Lists all JPG image files in the specified directory.
        2. Reads each image using OpenCV.
        3. Resizes each image to the specified target size.
        4. Scales the pixel values and converts the image to a format compatible with PIL.
        5. Converts each resized image to a PIL Image object.
        6. Appends each PIL Image object to a list.
        7. Returns the list of PIL Image objects.
        """

        all_images = []

        file_path = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if filename.endswith('.jpg')]

        for img in file_path:
            image = cv2.imread(img)
            resized_images=cv2.resize(image, target_size)
            resized_images = (resized_images * 255).astype(np.uint8)
            pil_images = Image.fromarray(resized_images)
            all_images.append(pil_images)

        return all_images

- The directoies of Spiral, Elliptical, Odd objectss images:

        - Spiral: /repository/Data/galaxy/image/cropped_spiral
        - Elliptical: /repository/Data/galaxy/image/cropped_elliptical
        - odd objects: /repository/Data/galaxy/image/cropped_odd

In [ ]:
sp_dir = '/path/to/repository/Data/galaxy/image/cropped_spiral'
el_dir = '/path/to/repository/Data/galaxy/image/cropped_elliptical'
odd_dir = '/path/to/repository/Data/galaxy/image/cropped_odd'

image_size = 200

sp_img = load_galaxy_images(sp_dir, target_size=(image_size,image_size))
el_img = load_galaxy_images(el_dir, target_size=(image_size,image_size))
odd_img = load_galaxy_images(odd_dir, target_size=(image_size,image_size))

all_data = np.concatenate([sp_img, el_img, odd_img])
np.shape(all_data)

- We define the **vision transformer** for both training and testing data sets: 

In [ ]:
# transforms for training data
train_transform = transforms.Compose([transforms.CenterCrop(image_size),
                                      transforms.RandomRotation(90),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0), ratio=(0.99, 1.01)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])


# transforms for test data
test_transform = transforms.Compose([transforms.CenterCrop(image_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])

### **CNN + vision transformer + original images**

- We use **"0"** for spiral class labels, **"1"** for elliptical class labels, and **"2"** for odd objects class labels.  

In [ ]:
label_s = [0] * len(sp_img)
label_e = [1] * len(el_img)
label_o = [2] * len(odd_img)

all_labels = np.concatenate([label_s, label_e, label_o])
len(all_labels)

- We split the data set into 75 percent traning set and 25 percent test set:

In [ ]:
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_data, all_labels, np.arange(len(all_labels)), 
                                                                                 test_size=0.25, shuffle=True, random_state=None)

y_train_encoded = to_categorical(y_train, num_classes=3)

- We apply the vision transformer for both training and testing samples:

In [ ]:
# Transformer for training data
transformed_X_train=[]
for i in range(len(X_train)):
  transformed_train_images = train_transform(X_train[i])
  new_image = np.transpose(transformed_train_images, (1, 2, 0))
  transformed_X_train.append(new_image)

# Transformer for testing data
transformed_X_test=[]
for j in range(len(X_test)):
  transformed_test_images = test_transform(X_test[j])
  new_images = np.transpose(transformed_test_images, (1, 2, 0))
  transformed_X_test.append(new_images)

- Since, the galaxy classifiers are unbalance class models, so we used the class weight in the program:

In [ ]:
class_weights = {0: len(all_data) / (3*len(sp_img)), 1: len(all_data) / (3*len(el_img)), 2: len(all_data) / (3*len(odd_img))}

In [ ]:
# input
x = Input(shape=(image_size,image_size,3))

#hidden layers
c0 = Conv2D(256, kernel_size=(3,3), strides=(1,1), padding="same")(x)
b0 = BatchNormalization()(c0)
m0 = MaxPooling2D(pool_size=(2, 2))(b0)
d0 = Dropout(0.1)(m0)

c1 = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding="same")(m0)
b1 = BatchNormalization()(c1)
m1 = MaxPooling2D(pool_size=(2, 2))(b1)
d1 = Dropout(0.1)(m1)

c2 = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding="same")(m1)
b2 = BatchNormalization()(c2)
m2 = MaxPooling2D(pool_size=(2, 2))(b2)
d2 = Dropout(0.1)(m2)

f = Flatten()(m2)

de0 = Dense(64, activation='relu')(f)
de1 = Dense(32, activation='relu')(de0)
de2 = Dense(3, activation='softmax')(de1)

model = Model(inputs=x, outputs=de2, name="cnn_transformer_galaxy_nonegalaxy")
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
model.summary()

- The **CNN** model uses EarlyStopping as callback function, batch size = 64, and number of epochs = 30 to fit the model on the training set:

In [ ]:
b_size =64
e_num = 30

# Callback Functions
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

history = model.fit(
np.array(transformed_X_train), y_train_encoded,
batch_size=b_size,
epochs=e_num,
verbose = 1,
class_weight=class_weights,
callbacks=es,
validation_split=0.1)

- Now, we apply the test set to examine the classification algorithm. Using the predicted label by the machine on original labels, we compute the elements of the confusion matrix.

In [ ]:
y_pred = model.predict(np.array(transformed_X_test))
y_pred_labels = np.argmax(y_pred, axis=1)

con = metrics.confusion_matrix(y_test, y_pred_labels)
print(con)

- To compare the performace of classifier with the random classifier, we calculate the **reciver operation charecterstic curve (ROC curve)**. The **area under the curve (AUC)** shows the probability of True positive rates of the classifier.

In [ ]:
y_score = gz2_training_model.predict_proba(X_test)  # Use predicted probabilities for ROC curve
y_test_bin = label_binarize(y_test, classes=list(range(3)))  # Binarize the true labels

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

class_names = ['Spiral', 'Elliptical', 'Odd objects']

fig, ax = plt.subplots(figsize=(10, 8))

mean_fpr = np.linspace(0, 1, 100)

for j in range(3):  # Iterate through each class
    tprs_interp = []
    for i in range(len(fpr)):
        tprs_interp.append(np.interp(mean_fpr, fpr[i][j], tpr[i][j]))

    mean_tpr = np.mean(tprs_interp, axis=0)
    mean_auc = np.trapz(mean_tpr, mean_fpr)

    ax.plot(mean_fpr, mean_tpr, label=f'{class_names[j]} (area = {mean_auc:.2f})', lw=2)

ax.plot([0, 1], [0, 1], linestyle='--', color='black', label='Random Guess')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title(f'Receiver Operating Characteristic (ROC) Curve')
ax.legend(loc='lower right')

plt.tight_layout()
plt.show()

- To measure the performance metrics of classifier, we compute **(Recall, Precision, f1_score, Accuracy, TSS(True Skill Statistic))**

In [ ]:
recall = recall_score(y_test, y_pred, average= 'weighted')
precision = precision_score(y_test, y_pred, average= 'weighted')
f1_score = f1_score(y_test, y_pred, average= 'weighted')
accuracy = accuracy_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tss=(tp/(tp+fn))-(fp/(fp+tn))

In [ ]:
print("Recall:", recall)
print("Precision:", precision)
print("F1_score:", f1_score)
print("Accuracy:", accuracy)
print("TSS:", tss)

### **ResNet50 + vision transformer + original images**

- We use **"0"** for spiral class labels, **"1"** for elliptical class labels, and **"2"** for odd objects class labels.  

In [ ]:
label_s = [0] * len(sp_img)
label_e = [1] * len(el_img)
label_o = [2] * len(odd_img)

all_labels = np.concatenate([label_s, label_e, label_o])
len(all_labels)

- We split the data set into 75 percent traning set and 25 percent test set:

In [ ]:
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_data, all_labels, np.arange(len(all_labels)), 
                                                                                 test_size=0.25, shuffle=True, random_state=None)

y_train_encoded = to_categorical(y_train, num_classes=3)

- We apply the vision transformer for both training and testing samples:

In [ ]:
# Transformer for training data
transformed_X_train=[]
for i in range(len(X_train)):
  transformed_train_images = train_transform(X_train[i])
  new_image = np.transpose(transformed_train_images, (1, 2, 0))
  transformed_X_train.append(new_image)

# Transformer for testing data
transformed_X_test=[]
for j in range(len(X_test)):
  transformed_test_images = test_transform(X_test[j])
  new_images = np.transpose(transformed_test_images, (1, 2, 0))
  transformed_X_test.append(new_images)

- Since, the galaxy classifiers are unbalance class models, so we used the class weight in the program:

In [ ]:
class_weights = {0: len(all_data) / (3*len(sp_img)), 1: len(all_data) / (3*len(el_img)), 2: len(all_data) / (3*len(odd_img))}

In [ ]:
# Defining the pretrained ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)  # The custom layers
output = Dense(2, activation='softmax')(x)  

model = Model(inputs=base_model.input, outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

history = model.fit(
np.array(transformed_X_train), y_train_encoded,
batch_size=b_size,
epochs=e_num,
verbose = 1,
callbacks=es,
class_weight=class_weights,
validation_split=0.1
)

- Now, we apply the test set to examine the classification algorithm. Using the predicted label by the machine on original labels, we compute the elements of the confusion matrix.

In [ ]:
y_pred = model.predict(np.array(transformed_X_test))
y_pred_labels = np.argmax(y_pred, axis=1)

con = metrics.confusion_matrix(y_test, y_pred_labels)
print(con)

- To compare the performace of classifier with the random classifier, we calculate the **reciver operation charecterstic curve (ROC curve)**. The **area under the curve (AUC)** shows the probability of True positive rates of the classifier.

In [ ]:
y_score = gz2_training_model.predict_proba(X_test)  # Use predicted probabilities for ROC curve
y_test_bin = label_binarize(y_test, classes=list(range(3)))  # Binarize the true labels

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

class_names = ['Spiral', 'Elliptical', 'Odd objects']

fig, ax = plt.subplots(figsize=(10, 8))

mean_fpr = np.linspace(0, 1, 100)

for j in range(3):  # Iterate through each class
    tprs_interp = []
    for i in range(len(fpr)):
        tprs_interp.append(np.interp(mean_fpr, fpr[i][j], tpr[i][j]))

    mean_tpr = np.mean(tprs_interp, axis=0)
    mean_auc = np.trapz(mean_tpr, mean_fpr)

    ax.plot(mean_fpr, mean_tpr, label=f'{class_names[j]} (area = {mean_auc:.2f})', lw=2)

ax.plot([0, 1], [0, 1], linestyle='--', color='black', label='Random Guess')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title(f'Receiver Operating Characteristic (ROC) Curve')
ax.legend(loc='lower right')

plt.tight_layout()
plt.show()

- To measure the performance metrics of classifier, we compute **(Recall, Precision, f1_score, Accuracy, TSS(True Skill Statistic))**

In [ ]:
recall = recall_score(y_test, y_pred, average= 'weighted')
precision = precision_score(y_test, y_pred, average= 'weighted')
f1_score = f1_score(y_test, y_pred, average= 'weighted')
accuracy = accuracy_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tss=(tp/(tp+fn))-(fp/(fp+tn))

In [ ]:
print("Recall:", recall)
print("Precision:", precision)
print("F1_score:", f1_score)
print("Accuracy:", accuracy)
print("TSS:", tss)

### **VGG16 + vision transformer + original images**

- We use **"0"** for spiral class labels, **"1"** for elliptical class labels, and **"2"** for odd objects class labels.  

In [ ]:
label_s = [0] * len(sp_img)
label_e = [1] * len(el_img)
label_o = [2] * len(odd_img)

all_labels = np.concatenate([label_s, label_e, label_o])
len(all_labels)

- We split the data set into 75 percent traning set and 25 percent test set:

In [ ]:
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_data, all_labels, np.arange(len(all_labels)), 
                                                                                 test_size=0.25, shuffle=True, random_state=None)

y_train_encoded = to_categorical(y_train, num_classes=3)

- We apply the vision transformer for both training and testing samples:

In [ ]:
# Transformer for training data
transformed_X_train=[]
for i in range(len(X_train)):
  transformed_train_images = train_transform(X_train[i])
  new_image = np.transpose(transformed_train_images, (1, 2, 0))
  transformed_X_train.append(new_image)

# Transformer for testing data
transformed_X_test=[]
for j in range(len(X_test)):
  transformed_test_images = test_transform(X_test[j])
  new_images = np.transpose(transformed_test_images, (1, 2, 0))
  transformed_X_test.append(new_images)

- Since, the galaxy classifiers are unbalance class models, so we used the class weight in the program:

In [ ]:
class_weights = {0: len(all_data) / (3*len(sp_img)), 1: len(all_data) / (3*len(el_img)), 2: len(all_data) / (3*len(odd_img))}

In [ ]:
# Defining the pretrained ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)  # The custom layers
output = Dense(2, activation='softmax')(x)  

model = Model(inputs=base_model.input, outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

history = model.fit(
np.array(transformed_X_train), y_train_encoded,
batch_size=b_size,
epochs=e_num,
verbose = 1,
callbacks=es,
class_weight=class_weights,
validation_split=0.1
)

- Now, we apply the test set to examine the classification algorithm. Using the predicted label by the machine on original labels, we compute the elements of the confusion matrix.

In [ ]:
y_pred = model.predict(np.array(transformed_X_test))
y_pred_labels = np.argmax(y_pred, axis=1)

con = metrics.confusion_matrix(y_test, y_pred_labels)
print(con)

- To compare the performace of classifier with the random classifier, we calculate the **reciver operation charecterstic curve (ROC curve)**. The **area under the curve (AUC)** shows the probability of True positive rates of the classifier.

In [ ]:
y_score = gz2_training_model.predict_proba(X_test)  # Use predicted probabilities for ROC curve
y_test_bin = label_binarize(y_test, classes=list(range(3)))  # Binarize the true labels

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

class_names = ['Spiral', 'Elliptical', 'Odd objects']

fig, ax = plt.subplots(figsize=(10, 8))

mean_fpr = np.linspace(0, 1, 100)

for j in range(3):  # Iterate through each class
    tprs_interp = []
    for i in range(len(fpr)):
        tprs_interp.append(np.interp(mean_fpr, fpr[i][j], tpr[i][j]))

    mean_tpr = np.mean(tprs_interp, axis=0)
    mean_auc = np.trapz(mean_tpr, mean_fpr)

    ax.plot(mean_fpr, mean_tpr, label=f'{class_names[j]} (area = {mean_auc:.2f})', lw=2)

ax.plot([0, 1], [0, 1], linestyle='--', color='black', label='Random Guess')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title(f'Receiver Operating Characteristic (ROC) Curve')
ax.legend(loc='lower right')

plt.tight_layout()
plt.show()

- To measure the performance metrics of classifier, we compute **(Recall, Precision, f1_score, Accuracy, TSS(True Skill Statistic))**

In [ ]:
recall = recall_score(y_test, y_pred, average= 'weighted')
precision = precision_score(y_test, y_pred, average= 'weighted')
f1_score = f1_score(y_test, y_pred, average= 'weighted')
accuracy = accuracy_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tss=(tp/(tp+fn))-(fp/(fp+tn))

In [ ]:
print("Recall:", recall)
print("Precision:", precision)
print("F1_score:", f1_score)
print("Accuracy:", accuracy)
print("TSS:", tss)